In [1]:
import xarray as xr

In [2]:
import pymongo as pym
import pandas

In [18]:
#client has access to a bunch of diff dbs and each db has a bunch of collections
#Mongo
#     ->Database
#         ->Collections
#db = database 
#col = collection

In [3]:
client = pym.MongoClient(
    'freedman-dev.app.arcc.albany.edu',
    27001,
    username='admin',
    password='Fr33dm@n'
)

In [4]:
#list all database names
client.list_database_names()

['admin', 'config', 'local']

In [5]:
#create new database and access the DB that we want
db=client['test1']
#create collection 
col=db['mycol']

In [6]:
db

Database(MongoClient(host=['freedman-dev.app.arcc.albany.edu:27001'], document_class=dict, tz_aware=False, connect=True), 'test1')

In [7]:
col

Collection(Database(MongoClient(host=['freedman-dev.app.arcc.albany.edu:27001'], document_class=dict, tz_aware=False, connect=True), 'test1'), 'mycol')

In [9]:
#create 1 record
col.insert_one({
    'name': "Arnold",
    'age': "25",
    'occupation': "software engineer"
})

In [13]:
#create many records
col.insert_many([{
    'name': "Arnold",
    'age': "25",
    'occupation': "software engineer"
},{
    'name': "Nick",
    'age': "29",
    'occupation': "data scientist"  
}])

In [14]:
#get all records
res=col.find({})

In [15]:
r=next(res)

In [17]:
res.distinct('name')
r

{'_id': ObjectId('5dd7fb91b52ed6b6b6de74c7'),
 'age': '29',
 'name': 'Nick',
 'occupation': 'data scientist'}

In [19]:
#what we want to find $eq is equal to 
res=col.find({
    'name': {'$eq': 'Nick'}
})

In [20]:
res.count()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


3

In [22]:
#iterate over res
r=next(res)
r['name']

'Nick'

In [24]:
#what we want to find $eq is equal to 
res2=col.find({
    'name': {'$in': ['Nick', 'Arnold']}
})

In [27]:
res2.count()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


5

In [28]:
#iterate over res2
r=next(res2)
r['name']

'Nick'

In [31]:
'''
ds = xr.open_dataset('somenetcdf')
{
'time': ,
'long': ,
'lat': ,
'name': 
}
'''
db=client['data']
col=db['records']


In [30]:
#populate a list of records that we will impot into database
#lat and long are 2 dimensional pairs instead of 1d
#1st is time so we squueze it out
#.values turns it into numpy and flatten turns it into 1d list
#zip will take take the 3 things in the params and put them into x,y,v 
# v is the point we are at and x,y are the lat and long
#we will create a list of records 
records=[]

for x,y,v in zip(
    ds.XLAT.squuze().values.flatten(), #2d numpy array and then flatten takes all the columns and stacks them into 1d
    ds.XLONG.squuze().values.flatten(),
    ds['U10'].squuze().values.flatten(),
)

    records.append({
        'time': pd.Timestamp('2017-10-30 12:00:000'),
        'long': float(y),
        'lat': float(x),
        'value': float(v),
        'name': 'U10'
    })
    
    #just created a list of records

#for ds['U10']

len(records)
#put the records into the collection
col.insert_many(records)

NameError: name 'ds' is not defined

In [32]:
#get all the records
recs=col.find({})

In [33]:
#get the record count
recs.count()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


61050

In [35]:
#we just have a curser to the data but dont have the data yet
#r=next(recs)
#taking every record inside the curser obj and putting it in data (list comprehention)
data=[r for r in recs]

In [36]:
data

[{'_id': ObjectId('5dd80410b52ed6b6b6e051bf'),
  'lat': 38.2445068359375,
  'long': -79.11739349365234,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 3.026827096939087},
 {'_id': ObjectId('5dd80410b52ed6b6b6e051c0'),
  'lat': 38.2445068359375,
  'long': -79.08148193359375,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 2.9267871379852295},
 {'_id': ObjectId('5dd80410b52ed6b6b6e051c1'),
  'lat': 38.2445068359375,
  'long': -79.04557037353516,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 3.10146427154541},
 {'_id': ObjectId('5dd80410b52ed6b6b6e051c2'),
  'lat': 38.2445068359375,
  'long': -79.00965118408203,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 3.062272787094116},
 {'_id': ObjectId('5dd80410b52ed6b6b6e051c3'),
  'lat': 38.2445068359375,
  'long': -78.97373962402344,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 2.67191672

In [37]:
recs=col.find({
    'lat': {'$gte': 42} #$ sign for mongo indexing
})

In [38]:
data=[r for r in recs]

In [39]:
recs.count()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


23375

In [40]:
data

[{'_id': ObjectId('5dd80410b52ed6b6b6e0e4e9'),
  'lat': 42.00514221191406,
  'long': -79.15331268310547,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 5.768162250518799},
 {'_id': ObjectId('5dd80410b52ed6b6b6e0e4ea'),
  'lat': 42.00514221191406,
  'long': -79.11739349365234,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 6.479415416717529},
 {'_id': ObjectId('5dd80410b52ed6b6b6e0e4eb'),
  'lat': 42.00514221191406,
  'long': -79.08148193359375,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 8.16376781463623},
 {'_id': ObjectId('5dd80410b52ed6b6b6e0e4ec'),
  'lat': 42.00514221191406,
  'long': -79.04557037353516,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 9.14024543762207},
 {'_id': ObjectId('5dd80410b52ed6b6b6e0e4ed'),
  'lat': 42.00514221191406,
  'long': -79.00965118408203,
  'name': 'U10',
  'time': datetime.datetime(2017, 10, 30, 12, 0),
  'value': 8.24385

In [ ]:
#structured this way because we are searching on time and records are in lat, long 